In [4]:
from pyspark import SparkContext
import ibmos2spark
import time
import boto3
import sys
import pprint
from pyspark.sql import Row
import xml.etree.ElementTree as ET
from pyspark.sql import SparkSession
import shutil
from io import BytesIO
import scandir
import json
import os
reload(sys)
sys.setdefaultencoding('utf-8')
sqlContext = SQLContext(sc)
urls=["anibis.ch",
                    "autoscout24.ch",
                    "objectif-moto",
                    "ldlc.be",
                    "annonces-automobile.com",
                    "artmajeur.com",
                    "mobilectrl.de/",
                    "feuerstuhl.net",
                    "tauschgnom.de",]
language_code={
           "french":"fr",
           "german":"de",
           "italian":'it',
           "japanese":"ja"
        }

credentials = {
            'endpoint': 's3-api.us-geo.objectstorage.softlayer.net',
            'access_key': '0b5378b16d334898ad3a30941640a18f',
            'secret_key': '4320f5522224a623439e3fc0613089d5628c044596abc6b5'
        }
#targetdir="/gpfs/global_fs01/sym_shared/YPProdSpark/user/sd1f-f9329d4abc7e63-0f85d1536d65/notebook/work/newsdata_"+time.strftime("%d-%m-%Y")+"/"
targetdir="/gpfs/global_fs01/sym_shared/YPProdSpark/user/sd1f-f9329d4abc7e63-0f85d1536d65/notebook/work/test/"
language="french"
bucket_name = 'blueumbrella-r1ix4buf-catalog-db12db8b'
cos = boto3.client('s3', endpoint_url="https://s3-api.us-geo.objectstorage.softlayer.net",aws_access_key_id=credentials['access_key'],aws_secret_access_key=credentials["secret_key"])
cos_res=boto3.resource('s3', endpoint_url="https://s3-api.us-geo.objectstorage.softlayer.net",aws_access_key_id=credentials['access_key'],aws_secret_access_key=credentials["secret_key"])
configuration_name = 'cos_config_string'  #you can give any string you like
cos_spark = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name=configuration_name)
spark = SparkSession.builder.getOrCreate()

def check_duplicate(language,type,date,keyword):
    path="nlu/"+"fr"+"/filtered/news/2017-10-06/headline_fr_news_f_20171006.csv"
    url=cos_spark.url(path,'blueumbrella-r1ix4buf-catalog-db12db8b')
    data=spark.read.format('com.databricks.spark.csv').options(header='true',inferSchema='true').load(url)
    data.registerTempTable('headline')
    cnt=sqlContext.sql('select ID from headline where news_headline_fr_news_f_20171006="'+keyword+'"').count()       
    return cnt

def make_manifest(code,description,tags,owner,dir):
    data= {}
    data["description"]="News Raw Data"
    data['language']=code
    data['tags']=[]
    for tag in tags:
        data['tags'].append(tag)
    data['owner']=owner
    json_data=json.dumps(data)
    outfile=open(dir+"manifest.json","w")
    json.dump(json_data,outfile,indent=2)
    
    
def load_raw_cos(date,language):
        code=language_code[language]
        #path="raw/"+code+"/news/webhose/common_"+code+"_news_r_"+date+"/data/"
        path="audio/test/"
        objects = cos.list_objects(Bucket=bucket_name,Prefix=path)
        file_lst=[]
        for obj in objects['Contents']:
            if ".xml" in obj["Key"]:
                file_lst.append(obj["Key"])
            
        for file_name in file_lst:
            print file_name
            content=cos.get_object(Bucket=bucket_name,Key=file_name)
            parse_xml(content["Body"].read(),file_name.split('/')[-1])

def parse_xml(content,file):
        #tree=ET.parse(content)
        #root=tree.getroot()
        root=ET.fromstring(content)
        language=root.find('language').text
        path=targetdir
        url=root.find('topic_url').text
        if not check_urls(url):
            path=targetdir+language+"/"
            title=root.find('discussion_title').text
            title_id=file[0:-4]+"_title"
            topic_id=file[0:-4]+"_topic"
            content=root.find('topic_text').text
            if title:
                if '"' in title:
                    title=title.replace('"',"'")
                if check_duplicate("french","headline","",title)<1:    
                    title_str='"'+str(title_id)+'","'+str(title)+'","'+language+'","'+"news_headline"+'"\n'
                    filename=path+"headline.csv"
                    f=open(filename,'a+')
                    f.write(title_str)
                if content:                                
                    if '"' in content:
                        content=content.replace('"',"'")
                content=filter_title(content,title,topic_id)
                if check_duplicate("french","article","",content)<1:
                    topic_str='"'+str(topic_id)+'","'+content+'","'+language+'","'+"news_article"+'"\n'
                    filename=path+"article.csv"
                    f=open(filename,'a+')
                    f.write(topic_str)
                i=1
                for post in root.findall('post'):
                    comment=post.text
                    if comment:
                        content1="".join([c if c.isalnum() else "" for c in content])
                        comment1="".join([c if c.isalnum() else "" for c in comment])
                        if content1!=comment1:
                            if '"' in comment:
                                comment=comment.replace('"',"'")
                            if check_duplicate("french","comment","",comment)<1:    
                                post_id=file[0:-4]+"_comment_"+str(i)
                                i+=1
                                post_str='"'+str(post_id)+'","'+str(comment)+'","'+language+'","'+"news_comment"+'"\n'
                                filename=path+"comment.csv"
                                f=open(filename,'a+')
                                f.write(post_str)
                            
def filter_title(article,title,id):
        if title:
            pos=article.find(title)                        
            if pos!=-1:
                #print id
                return article[pos+len(title):]
        return article   
    
def check_urls(url):
        for u in urls:
            if u in url:
                return True
        
        return False    
    
def setup():
        try:
            dr=targetdir+language
            shutil.rmtree(dr)
        except:
            pass
        os.makedirs(dr)
        for file in ["headline.csv","article.csv","comment.csv"]:
            f=open(dr+"/"+file,'a+')
            st="ID,news_"+file[:-4]+",language,data_source\n"
            f.write(st)    
    
def upload_filtered_COS():
        print "BEGIN UPLOAD: "

        for root,dirs,files in scandir.walk(targetdir):
            for d in dirs:
                language=language_code[d]
                path_to_upload="nlu/"+language+"/filtered/news/"+targetdir.split('_')[1]+"/"
                make_manifest(language,"News Filtered Data",path_to_upload.split("/"),"wuf@us.ibm.com",targetdir+d+"/")
                for root,dirs,files in scandir.walk(targetdir+d):
                    for file in files:
                        cos.upload_file(targetdir+d+"/"+file,"blueumbrella-r1ix4buf-catalog-db12db8b", path_to_upload+file)
                        os.remove(targetdir+d+"/"+file)
        print "END UPLOAD "   
setup()
load_raw_cos("","french") 
upload_filtered_COS()

In [9]:
ls

In [10]:
pwd

u'/gpfs/global_fs01/sym_shared/YPProdSpark/user/sd1f-f9329d4abc7e63-0f85d1536d65/notebook/work/test'

In [11]:
cd test

In [12]:
ls